In [ ]:
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation
import numpy as np
from mlp import MLP, Layer, InputLayer, OutputLayer, MultinoulliML
from utils import Adam

In [ ]:
def epoch_it(X, epochs, batchsize):
    n, _ = X.shape
    N = int(n/batchsize)
    for _ in range(epochs):
        for i in range(N):
            yield X[i*batchsize : (i+1)*batchsize]

def file_it(path, skiplines=0):
    with open(path) as f:
        for _ in range(skiplines):
            f.readline()
        yield from f

In [ ]:
class DropoutConfig:
    enabled = True
    p_hidden = 0.7
    p_input = 0.8

network = MLP(
    [
        InputLayer(28*28),
        Layer(100),
        Layer(100),
        Layer(100),
        Layer(100),
        OutputLayer(10)
    ],
    model=MultinoulliML(),
    optimizer=Adam(eps=0.1, clip_threshold=None),
    dropout=DropoutConfig
)

In [ ]:
batchsize = 20
epochs = 40

data = np.loadtxt("../data/train.csv", skiprows=1, delimiter=",", dtype=int)
valset = data[:100]
valset = (valset[:, 1:] / 255, valset[:, 0])
trainset = data[100:]
it = epoch_it(trainset, epochs, batchsize=batchsize)

network.init_training(batchsize=batchsize, validation_set=valset)

In [ ]:
for batch in it:
    X, Y = (batch[:,1:]/255, batch[:,0])
    cost, accuracy = network.train_minibatch((X, Y))
    print(accuracy, cost)
network.cleanup_training()

In [ ]:
it = map(
    lambda line: np.fromstring(line, sep=",", dtype=np.uint8),
    file_it("../data/test.csv", skiplines=1),
)

In [ ]:
data = next(it)
n = network.feedforward(data/255)
print(n)
plt.imshow(data.reshape((28,28)), cmap="gray", vmin = 0, vmax = 255)

In [ ]:
from main import Program

In [ ]:
%matplotlib qt
Program(data_path="../data/train.csv")